In [ ]:
pip install researchpy

In [ ]:
import researchpy as rp
import pandas as pd
import numpy as np
from scipy import stats
from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.regression.linear_model import OLS
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd

def ReplaceImprNonImpr(DataFrame, column):
    for i in DataFrame.index:
        if DataFrame.loc[i, column] == 'Improver':
            DataFrame.loc[i, column] = 1
        elif DataFrame.loc[i, column] == 'Non-improver':
            DataFrame.loc[i, column] = 0
        else:
            DataFrame.loc[i, column] = np.nan


##TO MODIFY BY USER
### Instruction: You will need to run thise codeonce for each Assessment number wanted (A0, A1, A2) 
#### Enter the directory with the cleaned csv file (folder with the file generated by 1_DataCleaner)
path = '/Users/malika/Documents/Five Lives/SI Data Analysis/LifestyleCognition/Datasets & Data cleaning/'
Data = pd.read_csv(path + 'DataframeA2-A0.csv')
Data = Data.set_index('participant_id')

DataCog = pd.read_csv(path + 'rate_of_change_per_game.csv')
DataCog = DataCog.set_index('user_id')


plot = '/Users/malika/Documents/Five Lives/SI Data Analysis/LifestyleCognition/HierarchicalClusteringAnalysis/A0A2 - IMP/Plots - Regression and clustering/'

In [ ]:
DataCog.columns

In [ ]:
cog = ['snap_change_category', 'snap_change_initial', 'snap_change_mean',
       'swift_acc_change', 'swift_RT_change', 'cast_acc_change',
       'cast_RT_change']

In [ ]:
DataCog.columns

In [ ]:
df2 = Data.join(DataCog)

## Many IDs still didn't complete the third assessment so need to first remove the nan variables
### by usiong one of the change variable from lifestyle

## Transform cognitive variables as a) 1, 0 instead of improver non-improver / b) float instead of strings


In [ ]:
df2

Clustering Analyisis
Great tutorial: https://stackabuse.com/hierarchical-clustering-with-python-and-scikit-learn/

In [ ]:
df2

In [ ]:
df.columns

In [ ]:
lifestyle = ['A0A2PA_duration', 'A0A2PA_intensity', 'A0A2PA_score',
       'A0A2smoking_situation', 'A0A2smoking_amount', 'A0A2socialI',
       'A0A2loneliness', 'A0A2sleep_problems', 'A0A2fruit_veg',
       'A0A2alcohol_unit', 'A0A2glasses_water', 'A0A2time_reading',
       'A0A2time_mental_stimulation', 'A0A2time_instrument', 'A0A2time_skill',
       'A0A2relaxed_scale', 'A0A2mood_rate', 'A0A2enjoyment_activities',
       'A0A2IMPPA_duration']

In [ ]:
df2[lifestyle]

## Lifestyle factors predicting cognition

In [ ]:
## A0PA_duration predicts A0snap_category_performance

LS factors with a significant change between A0 and A2:
socialI', p-value = 0.004512
Sleep_problems', P value = 0.000000
Glasses_water', P-value = 0.001318
time_instrument',P-value = 0.013119
Enjoyment_activities', P-value = 0.000883

LS factorss predicting improvements in cognittiton:
Alcohol, time skill, enjoyment activity but was bearly significant

Let's do the analysis for the 3 wiht good significant changes and more impactful: socialI, sleep and enjoyment activity


# Hierarchical Clustering
## 2 variables

sleep & social I

In [ ]:
df = df2[['A0A2sleep_problems', 'A0A2socialI' ]].dropna(axis=0)

In [ ]:
df

In [ ]:
from sklearn.preprocessing import StandardScaler
X = df.values
X= StandardScaler().fit_transform(X) # normalizing the features

In [ ]:
np.mean(X), np.std(X)

In [ ]:
#3 Using the dendrogram to find the optimal numbers of clusters. 
# First thing we're going to do is to import scipy library. scipy is #an open source Python library that contains tools to do #hierarchical clustering and building dendrograms. Only import the #needed tool.
import scipy.cluster.hierarchy as sch
#Lets create a dendrogram variable linkage is actually the algorithm #itself of hierarchical clustering and then in linkage we have to #specify on which data we apply and engage. This is X dataset
dendrogram = sch.dendrogram(sch.linkage(X, method  = "ward"))
plt.title('Dendrogram')
plt.xlabel('Customers')
plt.ylabel('Euclidean distances')
plt.show()

In [ ]:
#4 Fitting hierarchical clustering to the Mall_Customes dataset
# There are two algorithms for hierarchical clustering: #Agglomerative Hierarchical Clustering and 
# Divisive Hierarchical Clustering. We choose Euclidean distance and ward method for our algorithm class
from sklearn.cluster import AgglomerativeClustering 
hc = AgglomerativeClustering(n_clusters =3, affinity = 'euclidean', linkage ='ward')
# Lets try to fit the hierarchical clustering algorithm  to dataset #X while creating the clusters vector that tells for each customer #which cluster the customer belongs to.
y_hc=hc.fit_predict(X)

In [ ]:
y_hc

In [ ]:
len(X)

In [ ]:
#5 Visualizing the clusters. This code is similar to k-means #visualization code. We only replace the y_kmeans vector name to #y_hc for the hierarchical clustering
plt.scatter(X[y_hc==0, 0], X[y_hc==0, 1], s=100, c='red', label ='Cluster 1')
plt.scatter(X[y_hc==1, 0], X[y_hc==1, 1], s=100, c='blue', label ='Cluster 2')
plt.scatter(X[y_hc==2, 0], X[y_hc==2, 1], s=100, c='green', label ='Cluster 3')
#plt.scatter(X[y_hc==3, 0], X[y_hc==3, 1], s=100, c='cyan', label ='Cluster 4')
#plt.scatter(X[y_hc==4, 0], X[y_hc==4, 1], s=100, c='magenta', label ='Cluster 5')
plt.title('Clusters of Participants (Hierarchical Clustering Model)')
plt.xlabel('A0A2sleep_problems')
plt.ylabel('A0A2socialI')
plt.show()

In [ ]:
len(y_hc)

In [ ]:
df2.columns

In [ ]:
df['cluster_label'] = y_hc

In [ ]:
df

In [ ]:
df2['cast_RT_change'].isnull().sum()

In [ ]:
test = []
for i in cog:
    test.append(i)
    dfb = df.join(df2[i])
    dfb = dfb.dropna(axis=0)
    grpe0 = dfb[i][dfb['cluster_label']==0]
    grpe1 = dfb[i][dfb['cluster_label']==1]
    grpe2 = dfb[i][dfb['cluster_label']==2]
    #grpe3 = dfb[i][dfb['cluster_label']==3]
    f_val, p_val = stats.f_oneway(grpe0, grpe1, grpe2)  
    anova = "ANOVA results: F=", f_val, ", P =", p_val
    test.append(str(anova))

In [ ]:
test

In [ ]:
test

In [ ]:
title = dfa.columns[0] + ' & ' + dfa.columns[1]

In [ ]:
title

In [ ]:
## graph to do if time: separate dataset by improver / non improver 
## plot hue= cluster label for both 

In [ ]:
sns.catplot(data = dfa, x ='snap_words_change_A0_A2_category', hue = 'cluster_label', kind = 'count')

In [ ]:
title = dfa.columns[0] + ' & ' + dfa.columns[1]
x ='snap_words_change_A0_A2_category'
    #define mean and median
median = dfa[x].median()
mean = "%.2f" % dfa[x].mean() 
    #plot figures
plt.figure(figsize=(3, 5))
sns.boxplot(data = dfa, x = x, y = 'cluster_label', )


plt.legend(loc = 'upper right')
plt.title(title + ':A0snap_category_performance by group cluster')

In [ ]:
plt.figure(figsize=(3, 5))
sns.boxplot(data = dfa, x = 'cluster_label', y = 'A0snap_category_performance', color = 'pink')
plt.title(title + ':A0snap_category_performance by group cluster')

In [ ]:
dfb.groupby('cluster_label')['snap_words_change_A0_A2_category']

In [ ]:
dfb

In [ ]:
dfb.groupby('cluster_label')

In [ ]:
grpe0 = dfb['snap_words_change_A0_A2_category'][dfb['cluster_label']==0]
grpe1 = dfb['snap_words_change_A0_A2_category'][dfb['cluster_label']==1]
grpe2 = dfb['snap_words_change_A0_A2_category'][dfb['cluster_label']==2]


In [ ]:
grpe0

In [ ]:
f_val, p_val = stats.f_oneway(grpe0, grpe1, grpe2)  
 
print( "ANOVA results: F=", f_val, ", P =", p_val)  


### Social I  and enjoymentt activitiess

In [ ]:
df2.columns

In [ ]:
df = df2[['A0A2socialI', 'A0A2enjoyment_activities' ]].dropna(axis=0)

In [ ]:
from sklearn.preprocessing import StandardScaler
X = df.values
X= StandardScaler().fit_transform(X) # normalizing the features

In [ ]:
np.mean(X), np.std(X)

In [ ]:
#3 Using the dendrogram to find the optimal numbers of clusters. 
# First thing we're going to do is to import scipy library. scipy is #an open source Python library that contains tools to do #hierarchical clustering and building dendrograms. Only import the #needed tool.
import scipy.cluster.hierarchy as sch
#Lets create a dendrogram variable linkage is actually the algorithm #itself of hierarchical clustering and then in linkage we have to #specify on which data we apply and engage. This is X dataset
dendrogram = sch.dendrogram(sch.linkage(X, method  = "ward"))
plt.title('Dendrogram')
plt.xlabel('Customers')
plt.ylabel('Euclidean distances')
plt.show()

In [ ]:
#4 Fitting hierarchical clustering to the Mall_Customes dataset
# There are two algorithms for hierarchical clustering: #Agglomerative Hierarchical Clustering and 
# Divisive Hierarchical Clustering. We choose Euclidean distance and ward method for our algorithm class
from sklearn.cluster import AgglomerativeClustering 
hc = AgglomerativeClustering(n_clusters = 3, affinity = 'euclidean', linkage ='ward')
# Lets try to fit the hierarchical clustering algorithm  to dataset #X while creating the clusters vector that tells for each customer #which cluster the customer belongs to.
y_hc=hc.fit_predict(X)

#5 Visualizing the clusters. This code is similar to k-means #visualization code. We only replace the y_kmeans vector name to #y_hc for the hierarchical clustering
plt.scatter(X[y_hc==0, 0], X[y_hc==0, 1], s=100, c='red', label ='Cluster 1')
plt.scatter(X[y_hc==1, 0], X[y_hc==1, 1], s=100, c='blue', label ='Cluster 2')
plt.scatter(X[y_hc==2, 0], X[y_hc==2, 1], s=100, c='green', label ='Cluster 3')
#plt.scatter(X[y_hc==3, 0], X[y_hc==3, 1], s=100, c='cyan', label ='Cluster 4')
#plt.scatter(X[y_hc==4, 0], X[y_hc==4, 1], s=100, c='magenta', label ='Cluster 5')
plt.title('Clusters of Participants (Hierarchical Clustering Model)')
plt.xlabel('A0A2socialI')
plt.ylabel('A0A2enjoyment_activities')
plt.show()


In [ ]:
df['cluster_label'] = y_hc

In [ ]:
df

In [ ]:
test = []
for i in cog:
    test.append(i)
    dfb = df.join(df2[i])
    dfb = dfb.dropna(axis=0)
    grpe0 = dfb[i][dfb['cluster_label']==0]
    grpe1 = dfb[i][dfb['cluster_label']==1]
    grpe2 = dfb[i][dfb['cluster_label']==2]
    #grpe3 = dfb[i][dfb['cluster_label']==3]
    f_val, p_val = stats.f_oneway(grpe0, grpe1, grpe2)  
    anova = "ANOVA results: F=", f_val, ", P =", p_val
    test.append(str(anova))

In [ ]:
test

In [ ]:
test

In [ ]:
title = dfa.columns[0] + ' & ' + dfa.columns[1]

x ='A0snap_category_performance'
    #define mean and median
median = dfa[x].median()
mean = "%.2f" % dfa[x].mean() 
    #plot figures
plt.figure(figsize=(3, 5))
sns.displot(data = dfa, x = x, hue = 'cluster_label')
plt.axvline(dfa[x].median(), linestyle='dotted', color = 'magenta', label = f'median= {median}')
plt.axvline(dfa[x].mean(), linestyle='dotted', color = 'blue', label = f'mean= {mean}')


plt.legend(loc = 'upper right')
plt.title(title + ': A0snap_category_performance by group cluster')

In [ ]:
plt.figure(figsize=(3, 5))
sns.boxplot(data = dfa, x = 'cluster_label', y = 'A0snap_category_performance', color = 'pink')
plt.title('Time reading & Relaxed scale:A0snap_category_performance by group cluster')

In [ ]:
import pandas as pd
rp.summary_cont(dfa['cluster_label'])


In [ ]:
rp.summary_cont(dfa['A0snap_category_performance'].groupby(dfa['cluster_label']))


In [ ]:
grpe0 = dfa['A0snap_category_performance'][dfa['cluster_label']==0]
grpe1 = dfa['A0snap_category_performance'][dfa['cluster_label']==1]
grpe2 = dfa['A0snap_category_performance'][dfa['cluster_label']==2]

f_val, p_val = stats.f_oneway(grpe0, grpe1, grpe2)  
 
print( "ANOVA results: F=", f_val, ", P =", p_val)  

### time mental stim and relaxed scale

In [ ]:
df2.columns

In [ ]:
df = df2[['A0A2sleep_problems', 'A0A2enjoyment_activities']]
df = df.dropna(axis=0)

In [ ]:
df 

In [ ]:
from sklearn.preprocessing import StandardScaler
X = df.values
X= StandardScaler().fit_transform(X) # normalizing the features

In [ ]:
np.mean(X), np.std(X)

In [ ]:
#3 Using the dendrogram to find the optimal numbers of clusters. 
# First thing we're going to do is to import scipy library. scipy is #an open source Python library that contains tools to do #hierarchical clustering and building dendrograms. Only import the #needed tool.
import scipy.cluster.hierarchy as sch
#Lets create a dendrogram variable linkage is actually the algorithm #itself of hierarchical clustering and then in linkage we have to #specify on which data we apply and engage. This is X dataset
dendrogram = sch.dendrogram(sch.linkage(X, method  = "ward"))
plt.title('Dendrogram')
plt.xlabel('Customers')
plt.ylabel('Euclidean distances')
plt.show()

In [ ]:
#4 Fitting hierarchical clustering to the Mall_Customes dataset
# There are two algorithms for hierarchical clustering: #Agglomerative Hierarchical Clustering and 
# Divisive Hierarchical Clustering. We choose Euclidean distance and ward method for our algorithm class
from sklearn.cluster import AgglomerativeClustering 
hc = AgglomerativeClustering(n_clusters = 3, affinity = 'euclidean', linkage ='ward')
# Lets try to fit the hierarchical clustering algorithm  to dataset #X while creating the clusters vector that tells for each customer #which cluster the customer belongs to.
y_hc=hc.fit_predict(X)

#5 Visualizing the clusters. This code is similar to k-means #visualization code. We only replace the y_kmeans vector name to #y_hc for the hierarchical clustering
plt.scatter(X[y_hc==0, 0], X[y_hc==0, 1], s=100, c='red', label ='Cluster 1')
plt.scatter(X[y_hc==1, 0], X[y_hc==1, 1], s=100, c='blue', label ='Cluster 2')
plt.scatter(X[y_hc==2, 0], X[y_hc==2, 1], s=100, c='green', label ='Cluster 3')
#plt.scatter(X[y_hc==3, 0], X[y_hc==3, 1], s=100, c='cyan', label ='Cluster 4')
#plt.scatter(X[y_hc==4, 0], X[y_hc==4, 1], s=100, c='magenta', label ='Cluster 5')
plt.title('Clusters of Participants (Hierarchical Clustering Model)')
plt.xlabel('A0A2sleep_problems')
plt.ylabel('A2enjoyment_activities')
plt.show()

In [ ]:
df['cluster_label'] = y_hc

In [ ]:
df

In [ ]:
test = []
for i in cog:
    test.append(i)
    dfb = df.join(df2[i])
    dfb = dfb.dropna(axis=0)
    grpe0 = dfb[i][dfb['cluster_label']==0]
    grpe1 = dfb[i][dfb['cluster_label']==1]
    grpe2 = dfb[i][dfb['cluster_label']==2]
    f_val, p_val = stats.f_oneway(grpe0, grpe1, grpe2)  
    anova = "ANOVA results: F=", f_val, ", P =", p_val
    test.append(str(anova))

In [ ]:
test

In [ ]:
# Significance for snap_words_change_A0_A2_initial and swift_RT_change_A0_A2

In [ ]:
path = '/Users/malika/Documents/Five Lives/SI Data Analysis/LifestyleCognition/Datasets & Data cleaning/'
Data = pd.read_csv(path + 'DataframeA2-A0.csv')

In [ ]:
dfa.to_csv(plot + 'df_clustering')

In [ ]:
sns.catplot(data = dfa, x ='snap_words_change_A0_A2_initial', hue = 'cluster_label', kind = 'count')

In [ ]:
sns.catplot(data = dfa, x ='swift_RT_change_A0_A2', hue = 'cluster_label', kind = 'count')

In [ ]:
plt.figure(figsize=(3, 5))
sns.boxplot(data = dfa, x = 'cluster_label', y = 'A0snap_category_performance', color = 'pink')
plt.title(title + ': A0snap_category_performance by group cluster')

In [ ]:
rp.summary_cont(dfa['snap_words_change_A0_A2_initial'].groupby(dfa['cluster_label']))

In [ ]:
test = []
for i in cog:
    test.append(i)
    dfb = df.join(df2[i])
    dfb = dfb.dropna(axis=0)
    grpe0 = dfb[i][dfb['cluster_label']==0]
    grpe1 = dfb[i][dfb['cluster_label']==1]
    grpe2 = dfb[i][dfb['cluster_label']==2]
    f_val, p_val = stats.f_oneway(grpe0, grpe1, grpe2)  
    anova = "ANOVA results: F=", f_val, ", P =", p_val
    test.append(str(anova))

In [ ]:
dfb = df.join(df2['snap_words_change_A0_A2_initial'])
dfb = dfb.dropna(axis=0)

In [ ]:
stats.multicomp.pairwise_tukeyhsd

In [ ]:
# Tukey test
tukey = pairwise_tukeyhsd(endog=dfb['snap_words_change_A0_A2_initial'],
                          groups=dfb['cluster_label'],
                          alpha=0.05)

In [ ]:
print('snap_words_change_A0_A2_initial', tukey)

In [ ]:
dfb = df.join(df2['swift_RT_change_A0_A2'])
dfb = dfb.dropna(axis=0)

In [ ]:
# Tukey test
tukey = pairwise_tukeyhsd(endog=dfb['swift_RT_change_A0_A2'],
                          groups=dfb['cluster_label'],
                          alpha=0.05)

In [ ]:
print('swift_RT_change_A0_A2', tukey)

In [ ]:
df_diff = dfa[dfa['cluster_label']!=1]

In [ ]:
title = df_diff.columns[0] + ' & ' + dfa.columns[1]

x ='A0snap_category_performance'
    #define mean and median
median = df_diff[x].median()
mean = "%.2f" % dfa[x].mean() 
    #plot figures
plt.figure(figsize=(3, 5))
sns.displot(data = df_diff, x = x, hue = 'cluster_label', palette = ['grey', 'yellow'])
plt.axvline(df_diff[x].median(), linestyle='dotted', color = 'magenta', label = f'median= {median}')
plt.axvline(df_diff[x].mean(), linestyle='dotted', color = 'blue', label = f'mean= {mean}')


plt.legend(loc = 'upper right')
plt.title(title + ': A0snap_category_performance by group cluster')